In [1]:
pip install Pillow==2.2.2

In [14]:
!pip install tensorflow==1.12.0
import tensorflow as tf
print(tf.__version__)

ERROR! Session/line number was not unique in database. History logging moved to new session 60
     |████████████████████████████████| 83.1MB 71kB/s 
     |████████████████████████████████| 3.1MB 19.4MB/s 
  Found existing installation: tensorboard 2.1.1
    Uninstalling tensorboard-2.1.1:
      Successfully uninstalled tensorboard-2.1.1
  Found existing installation: tensorflow 2.2.0rc1
    Uninstalling tensorflow-2.2.0rc1:
      Successfully uninstalled tensorflow-2.2.0rc1


2.2.0-rc1


In [0]:
import glob
import os

import numpy as np
from keras import Input
from keras.applications import VGG19
from keras.callbacks import TensorBoard
from keras.layers import BatchNormalization, Activation, LeakyReLU, Add, Dense, PReLU, Flatten
from keras.layers.convolutional import Conv2D, UpSampling2D
from keras.models import Model
from keras.optimizers import Adam
from keras_preprocessing.image import img_to_array, load_img
import cv2

Method to create Residual Block
* The output of the pre-residual block goes to the first residual block. The output of the first residual block goes to the second residual block, and so on, up to the 16th residual block.

In [0]:
def residual_block(x):
    """
    Residual block
    """
    filters = [64, 64]
    kernel_size = 3
    strides = 1
    padding = "same"
    momentum = 0.8
    activation = "relu"

    res = Conv2D(filters=filters[0], kernel_size=kernel_size, 
                 strides=strides, padding=padding)(x)
    res = Activation(activation=activation)(res)
    res = BatchNormalization(momentum=momentum)(res)

    res = Conv2D(filters=filters[1], kernel_size=kernel_size, 
                 strides=strides, padding=padding)(res)
    res = BatchNormalization(momentum=momentum)(res)

    # Add res and x
    res = Add()([res, x])
    return res

## The Generator Network
* Defining hyperparameters
* Gen1 - Creating Input layers

##### Model Architecture
* Gen2 -Add the post-residual block (a 2D conv layer followed by a batch normalization layer)
* Gen3 -Add an Add layer to take the sum of the output from the pre-residual block, which is gen1, and the output from the post-residual block, which is Gen2.
* Gen4 & Gen5 -add up the unsampling block
* Gen6 -Output Conv-layer

In [0]:
def Build_Generator():

  residual_blocks = 16
  momentum = 0.8
  input_shape = (64, 64, 3)

  input_layer = Input(shape=input_shape)
  gen1 = Conv2D(filters=64, kernel_size=9, strides=1, padding='same', activation='relu')(input_layer)

  res = residual_block(gen1)
  for i in range(residual_blocks - 1):
    res = residual_block(res)

  
  gen2 = Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(res)
  gen2 = BatchNormalization(momentum=momentum)(gen2)
  gen3 = Add()([gen2, gen1]) # This layer generates another tensor of similar shape.


  gen4 = UpSampling2D(size=2)(gen3)
  gen4 = Conv2D(filters=256, kernel_size=3, strides=1, padding='same')(gen4)
  gen4 = Activation('relu')(gen4)


  gen5 = UpSampling2D(size=2)(gen4)
  gen5 = Conv2D(filters=256, kernel_size=3, strides=1, padding='same')(gen5)
  gen5 = Activation('relu')(gen5)

  gen6 = Conv2D(filters=3, kernel_size=9, strides=1, padding='same')(gen5)
  output = Activation('tanh')(gen6)

  model_gen = Model(inputs=[input_layer], outputs=[output], name='generator') 

  return model_gen


## The Discriminator Network
____



### Configuration:
* hyperparameters required for the discriminator network
* Dist - Adding a convolution block.
* Dist1-Dist8 are 8 convolution blocks.
* Dist9 -Add a dense layer with 1,024 nodes

In [0]:
def Build_Discriminator():
  # Introducing Hyperparameter Requires.
  leakyrelu_alpha = 0.2   
  momentum = 0.8
  input_shape = (256, 256, 3)

  input_layer = Input(shape=input_shape) # Model's Input Layer. 


  dis1 = Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(input_layer)
  dis1 = LeakyReLU(alpha=leakyrelu_alpha)(dis1)
  print("Dist1 is created..")


  dis2 = Conv2D(filters=64, kernel_size=3, strides=2, padding='same')(dis1)
  dis2 = LeakyReLU(alpha=leakyrelu_alpha)(dis2)
  dis2 = BatchNormalization(momentum=momentum)(dis2)
  print("Dist2 is created..")


  dis3 = Conv2D(filters=128, kernel_size=3, strides=1, padding='same')(dis2)
  dis3 = LeakyReLU(alpha=leakyrelu_alpha)(dis3)
  dis3 = BatchNormalization(momentum=momentum)(dis3)
  print("Dist3 is created..")


  dis4 = Conv2D(filters=128, kernel_size=3, strides=2, padding='same')(dis3)
  dis4 = LeakyReLU(alpha=leakyrelu_alpha)(dis4)
  dis4 = BatchNormalization(momentum=0.8)(dis4)
  print("Dist4 is created..")


  dis5 = Conv2D(256, kernel_size=3, strides=1, padding='same')(dis4)
  dis5 = LeakyReLU(alpha=leakyrelu_alpha)(dis5)
  dis5 = BatchNormalization(momentum=momentum)(dis5)
  print("Dist5 is created..")


  dis6 = Conv2D(filters=256, kernel_size=3, strides=2, padding='same')(dis5)
  dis6 = LeakyReLU(alpha=leakyrelu_alpha)(dis6)
  dis6 = BatchNormalization(momentum=momentum)(dis6)
  print("Dist6 is created..")


  dis7 = Conv2D(filters=512, kernel_size=3, strides=1, padding='same')(dis6)
  dis7 = LeakyReLU(alpha=leakyrelu_alpha)(dis7)
  dis7 = BatchNormalization(momentum=momentum)(dis7)
  print("Dist7 is created..")

  dis8 = Conv2D(filters=512, kernel_size=3, strides=2, padding='same')(dis7)
  dis8 = LeakyReLU(alpha=leakyrelu_alpha)(dis8)
  dis8 = BatchNormalization(momentum=momentum)(dis8)
  print("Dist8 is created..")

  dis9 = Dense(units=1024)(dis8) #Nodes = 1024
  dis9 = LeakyReLU(alpha=0.2)(dis9)
  output = Dense(units=1, activation='sigmoid')(dis9) # Dense layer to return probabilities.
  print("Dense layer is created..")

  model_dist = Model(inputs=[input_layer], outputs=[output], 
                name='discriminator')

  print("Discriminator Model created..")
  return model_dist


## The adversarial network
The adversarial network is a combined network that uses the generator, the discriminator and VGG19.


____
###Configuration:
* 1 -creating an input layer of the network 
** The adversarial network will receive an image of a shape of (64, 64, 3), which is why we have created an input layer.
* 2 -generate fake high-resolution images using the generator network
* 3 -extract the features of the fake images using the VGG19 network
* 4 -make the discriminator network non-trainable in the adversarial network.
** We are making the discriminator network non-trainable because we don’t want to train the discriminator network while we train the generator network.

* 5 -pass the fake images to the discriminator network.

In [0]:
def build_adversarial_model(generator, discriminator, vgg):

    #1
    input_low_resolution = Input(shape=(64, 64, 3))

    #2
    fake_hr_images = generator(input_low_resolution)
    #3
    fake_features = vgg(fake_hr_images)

    #4
    discriminator.trainable = False

    #5
    output = discriminator(fake_hr_images)
    
    # Creating Model.
    model_adv = Model(inputs=[input_low_resolution], outputs=[output, fake_features])

    for layer in model.layers:
        print(layer.name, layer.trainable)

    print(model.summary())
    return model_adv

## Training the SRGAN

Training the SRGAN network is a two-step process. In the first step, we train the discriminator network. In the second step, we train the adversarial network, which eventually trains the generator network.

In [0]:
data_dir = ""
epochs = 20000
batch_size = 1

# Shape of low-resolution and high-resolution images
low_resolution_shape = (64, 64, 3)
high_resolution_shape = (256, 256, 3)